In [5]:
import numpy as np
from PIL import Image
import ctypes
import matplotlib.pyplot as plt
import random
import os

In [6]:
directory_inputs = ["../database/16x16/asie_sud_est","../database/16x16/rome_grece","../database/16x16/chateau_europe"]
directory_test_inputs = ["../database/16x16/tests/asie_sud_est","../database/16x16/tests/rome_grece","../database/16x16/tests/chateau_europe"]
# directory_inputs = ["../database/resized_dataset/amerique_sud","../database/resized_dataset/asie_sud_est","../database/resized_dataset/rome_grece"]

In [7]:
X = []
Y = []

for i in range(len(directory_inputs)):
    for image_file in os.listdir(directory_inputs[i]):
        if 'image' in image_file:
            if i == 0 :
                image_file = Image.open(directory_inputs[i]+'/'+image_file)
                # print(image_file)
                X.append(np.asarray(image_file))
                Y.append([1., -1., -1.])
            elif i == 1 :
                image_file = Image.open(directory_inputs[i]+'/'+image_file)
                # print(image_file)
                X.append(np.asarray(image_file))
                Y.append([-1., 1., -1.])
            elif i == 2 :
                image_file = Image.open(directory_inputs[i]+'/'+image_file)
                # print(image_file)
                X.append(np.asarray(image_file))
                Y.append([-1.,-1.,1.])

X = np.array(X, dtype=ctypes.c_float)
X_flat = X.flatten()/255 * 2 - 1

Y = np.array(Y, dtype=ctypes.c_float)
Y_flat = Y.flatten()

In [8]:
X_train = []
Y_train = []

for i in range(len(directory_test_inputs)):
    for image_file in os.listdir(directory_test_inputs[i]):
        if 'image' in image_file:
            if i == 0 :
                image_file = Image.open(directory_test_inputs[i]+'/'+image_file)
                # print(image_file)
                X_train.append(np.asarray(image_file))
                Y_train.append([1., -1.,-1.])
            elif i == 1 :
                image_file = Image.open(directory_test_inputs[i]+'/'+image_file)
                # print(image_file)
                X_train.append(np.asarray(image_file))
                Y_train.append([-1., 1.,-1.])
            elif i == 2 :
                image_file = Image.open(directory_test_inputs[i]+'/'+image_file)
                # print(image_file)
                X_train.append(np.asarray(image_file))
                Y_train.append([-1., -1.,1.])

X_train = np.array(X_train, dtype=ctypes.c_float)
X_train_flat = X_train.flatten()/255 * 2 - 1

Y_train = np.array(Y_train, dtype=ctypes.c_float)
Y_train_flat = Y_train.flatten()

In [9]:
my_lib = ctypes.CDLL(r"..\rust_lib\target\release\rust_lib.dll")

my_lib.delete_float_array.argtypes = [
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int32,
]

my_lib.delete_float_array.restype = None

In [10]:
LP_c_float = ctypes.POINTER(ctypes.c_float)
x_to_predict = (ctypes.c_float * len(X_flat))(*X_flat)
x_to_predict_ptr = ctypes.cast(x_to_predict, LP_c_float)
LP_c_float = ctypes.POINTER(ctypes.c_float)
y_to_predict = (ctypes.c_float * len(Y_flat))(*Y_flat)
y_to_predict_ptr = ctypes.cast(y_to_predict, LP_c_float)

LP_c_float = ctypes.POINTER(ctypes.c_float)
x_to_predict_test = (ctypes.c_float * len(X_train_flat))(*X_train_flat)
x_to_predict_test_ptr = ctypes.cast(x_to_predict_test, LP_c_float)
# LP_c_float = ctypes.POINTER(ctypes.c_float)
# y_to_predict_test = (ctypes.c_float * len(Y_train_flat))(*Y_train_flat)
# y_to_predict_test_ptr = ctypes.cast(y_to_predict_test, LP_c_float)

my_lib.radial_basis_function_model.argtypes = [
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int32,
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int32,
    ctypes.c_int32,
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int32,
    ctypes.c_float,
    ctypes.c_bool,
    ctypes.c_int32,
    ctypes.c_bool,
]

my_lib.radial_basis_function_model.restype = ctypes.POINTER(ctypes.c_float)

In [ ]:
rbf_trained_model_ptr = my_lib.radial_basis_function_model(
    x_to_predict_ptr,  # pointer_to_inputs_train : *mut f32,
    len(X),  # number_of_training_inputs : usize,
    x_to_predict_test_ptr,  # pointer_to_inputs_to_predict : *mut f32,
    len(x_to_predict),  # number_of_inputs_to_predict : usize,
    len(X[0]),  # dimension_of_inputs : usize,
    y_to_predict_ptr,  # pointer_to_labels : *mut f32,
    3,  # number_of_classes : usize,
    0.5,  # gamma : f32,
    True,  # is_classification : bool,
    50,  # number_of_clusters : usize
    False,
)

rbf_trained_model = np.ctypeslib.as_array(rbf_trained_model_ptr, (len(x_to_predict) * 3,))

print(len(rbf_trained_model))

In [ ]:
image_a_predire_grec = np.asarray(Image.open('test_images/predict_grec.jpg').resize((16,16)).convert('RGB'))
image_a_predire_asie = np.asarray(Image.open('test_images/predict_asie.jpg').resize((16,16)).convert('RGB'))
image_a_predire_asie_maps = np.asarray(Image.open('test_images/predict_asia_maps.png').resize((16,16)).convert('RGB'))

image_a_predire_data = np.array([image_a_predire_asie,image_a_predire_grec,image_a_predire_asie_maps],dtype=ctypes.c_float)
image_a_predire_data_flat=image_a_predire_data.flatten()



In [ ]:
for j in range(0,len(rbf_trained_model)-2,3) :
    if rbf_trained_model[j] > rbf_trained_model[j+1] and rbf_trained_model[j] > rbf_trained_model[j+2] :
        print("L'image a deviner vient asie")
    elif rbf_trained_model[j+1] > rbf_trained_model[j] and rbf_trained_model[j+1] > rbf_trained_model[j+2] :
        print("L'image a deviner vient rome")
    elif rbf_trained_model[j+2] > rbf_trained_model[j] and rbf_trained_model[j+2] > rbf_trained_model[j+1]:
        print("L'image a deviner vient de chateau")
    else :
        print('résultats non concluants')

In [ ]:
print(rbf_trained_model[0:15])